In [1]:
import pandas as pd

In [2]:
df_1976 = pd.read_csv('/mnt/hdd01/patentsview/Fulltext Data/Description/g_detail_desc_text_1976.tsv.zip', compression='zip', sep='\t', header=0)

In [3]:
df_1976.head()

,patent_id,description_text,description_length
0,3948708,Referring now more particularly to the drawing...,2142
1,3995386,"In the embodiment illustrated by FIGS. 1 - 3, ...",1196
2,3944872,"Referring now to FIGS. 1 and 2, a magnetron st...",3239
3,3974938,The embodiment seen in these Figures is made u...,1331
4,3931106,The invention is now illustrated by the follow...,3589


I want to write data to a postgres database from tsv files, which are stored in one directory on my server. The database is already existing and I want to link the data by appln_id which is not already present in the files. The data has a column named text and a column named patent_id. The patent_id can be linked to the existing table tls211_pat_publn where patent_id can be matched with publn_nr, on the condition that publn_auth = "US". The corresponding appln_id can then be found in the row and used to link the new table to the existing database. Also make a new column in the table with publn_auth = "US". Provide me the Python code using the library SQLAlchemy.